In [1]:
import pandas as pd
from pandas import Series, DataFrame # 最常用的类

In [2]:
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.set_printoptions(precision=4, suppress=True) # numpy 不要使用科学计数法，小数4位

对数据集进行**分类**，**并在每一组上应用一个聚合函数或转换函数**，这通常是数据分析工作流中的一个重要部分

在**载入，合并，准备数据集之后，可能需要计算分组统计或者数据透视表**用于报告或可视化的目的，`pandas`提供了一个灵活的`groupby`接口，允许以**一种自然的方式对数据集进行切片，切块，总结**

类似SQL中的聚合功能

## GroupBy机制

Hadley Wuckham 是许多流行R语言软件包的作者，它创建了用于描述组操作的术语**拆分-应用-联合(`split-apply-combine`)**

1. 在操作的第一步： 数据包含在pandas对象中，可以是Series，DataFrame或其他数据结构，之后**根据你提供的一个或多个键分离到各个组中**
    - 分离操作是在数据对象的特定轴向上进行的
2. 分组操作后，**一个函数就可以应用到各个组中**，产生新的值
3. 最终，**所有的函数应用结果会联合为一个新的结果对象**， 结果对象的形式通常取决于对数据进行的操作

![split-apply-combine](https://gitee.com/zhao_long/image-store/raw/master/img/20201116185314.png)

**分组聚合**
- **分组键可是多种形式的，并且键不一定是完全相同的类型**
    1. 与需要分组的轴向长度一致的**值列表或值数组**
    2. DataFrame的**列名**的值
    3. 可以将分组轴向上的值和分组名称相匹配的**字典或Series**
    4. 可以在轴索引或索引中的单个标签上调用的**函数**
- 后面的三种方法是可以**产生用于分隔对象的值数组**的快捷方式

![四种分组](https://gitee.com/zhao_long/image-store/raw/master/img/20201116205149.png)

In [3]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


In [9]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [12]:
grouped.head()

0   -0.204708
1    0.478943
2   -0.519439
3   -0.555730
4    1.965781
Name: data1, dtype: float64

返回一个`GriupBy`对象，**除了一些关于分组键`df['key1]`的一些中间数据外，实际上还没有进行任何计算**。这个对象拥有所有必须的信息，之后**可以在每一分租上应用一些操作**

In [14]:
list(grouped)

[('a',
  0   -0.204708
  1    0.478943
  4    1.965781
  Name: data1, dtype: float64),
 ('b',
  2   -0.519439
  3   -0.555730
  Name: data1, dtype: float64)]

In [15]:
dict(list(grouped))

{'a': 0   -0.204708
 1    0.478943
 4    1.965781
 Name: data1, dtype: float64,
 'b': 2   -0.519439
 3   -0.555730
 Name: data1, dtype: float64}

In [16]:
grouped.mean()

key1
a    0.746672
b   -0.537585
Name: data1, dtype: float64

In [18]:
means = df['data1'].groupby( [df['key1'], df['key2']])
means

In [19]:
list(means)

[(('a', 'one'),
  0   -0.204708
  4    1.965781
  Name: data1, dtype: float64),
 (('a', 'two'),
  1    0.478943
  Name: data1, dtype: float64),
 (('b', 'one'),
  2   -0.519439
  Name: data1, dtype: float64),
 (('b', 'two'),
  3   -0.55573
  Name: data1, dtype: float64)]

In [20]:
means.mean()

key1  key2
a     one     0.880536
      two     0.478943
b     one    -0.519439
      two    -0.555730
Name: data1, dtype: float64

In [21]:
means.mean().unstack()

key2,one,two
key1,,
a,0.880536,0.478943
b,-0.519439,-0.555730


In [22]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

In [24]:
df['data1'].groupby([states, years]).mean()

California  2005    0.478943
            2006   -0.519439
Ohio        2005   -0.380219
            2006    1.965781
Name: data1, dtype: float64

In [25]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.746672,0.910916
b,-0.537585,0.525384


In [26]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.880536  1.319920
     two   0.478943  0.092908
b    one  -0.519439  0.281746
     two  -0.555730  0.769023

In [27]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [28]:
df.groupby( 'key2').size()

key2
one    3
two    2
dtype: int64

### 遍历各分组

`GroupBy`对象**支持迭代**，会生成一个**包含组名和数据块的2维元素**

In [31]:
for key, value in df.groupby('key1'):
    print(key,value, sep="---->\n", end='\n\n')

a---->
  key1 key2     data1     data2
0    a  one -0.204708  1.393406
1    a  two  0.478943  0.092908
4    a  one  1.965781  1.246435

b---->
  key1 key2     data1     data2
2    b  one -0.519439  0.281746
3    b  two -0.555730  0.769023



In [33]:
for key, value in df.groupby(['key1', 'key2']):
    print(key,value, sep="---->\n", end='\n\n')

('a', 'one')---->
  key1 key2     data1     data2
0    a  one -0.204708  1.393406
4    a  one  1.965781  1.246435

('a', 'two')---->
  key1 key2     data1     data2
1    a  two  0.478943  0.092908

('b', 'one')---->
  key1 key2     data1     data2
2    b  one -0.519439  0.281746

('b', 'two')---->
  key1 key2    data1     data2
3    b  two -0.55573  0.769023



In [38]:
dict(list(df.groupby(['key1', 'key2'])))

{('a',
  'one'):   key1 key2     data1     data2
 0    a  one -0.204708  1.393406
 4    a  one  1.965781  1.246435,
 ('a',
  'two'):   key1 key2     data1     data2
 1    a  two  0.478943  0.092908,
 ('b',
  'one'):   key1 key2     data1     data2
 2    b  one -0.519439  0.281746,
 ('b',
  'two'):   key1 key2    data1     data2
 3    b  two -0.55573  0.769023}

In [39]:
dict(list(df.groupby(['key1', 'key2'])))[('a','one')]

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
4,a,one,1.965781,1.246435


In [41]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [43]:
grouped = df.groupby(df.dtypes, axis=1)

In [44]:
for key,value in grouped:
    print(key)
    print('----------------------------------------')
    print(value)

float64
----------------------------------------
      data1     data2
0 -0.204708  1.393406
1  0.478943  0.092908
2 -0.519439  0.281746
3 -0.555730  0.769023
4  1.965781  1.246435
object
----------------------------------------
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### 选择一列或所有列的子集

In [45]:
df.groupby('key1')['data1']

In [46]:
df.groupby('key1')[['data1']]

In [47]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   1.319920
     two   0.092908
b    one   0.281746
     two   0.769023

In [48]:
df.groupby(['key1', 'key2'])['data2'].mean()

key1  key2
a     one     1.319920
      two     0.092908
b     one     0.281746
      two     0.769023
Name: data2, dtype: float64

### 使用字典和Series分组

**分组信息可能以非数组形式存在**

In [50]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2, [1,2]] =np.nan

In [51]:
people

,a,b,c,d,e
Joe,0.560145,-1.265934,0.119827,-1.063512,0.332883
Steve,-2.359419,-0.199543,-1.541996,-0.970736,-1.307030
Wes,0.286350,NaN,NaN,0.331286,1.349742
Jim,0.069877,0.246674,-0.011862,1.004812,1.327195
Travis,-0.919262,-1.549106,0.022185,0.758363,-0.660524


In [52]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}
mapping

{'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}

In [55]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,-0.943685,-0.372906
Steve,-2.512731,-3.865992
Wes,0.331286,1.636092
Jim,0.992950,1.643745
Travis,0.780548,-3.128892


In [57]:
by_column.count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


![颜色映射](https://gitee.com/zhao_long/image-store/raw/master/img/20201116205937.png)

In [58]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [60]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 使用函数分组

**使用Python函数是定义分组关系的一种更为通用的方式**

**作为分组键传递的函数将会按照每个索引值调用一次，同时返回值被用作分组名称**

In [61]:
people.groupby(len).count()

,a,b,c,d,e
3,3,2,2,3,3
5,1,1,1,1,1
6,1,1,1,1,1


In [62]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one  0.286350 -1.265934  0.119827 -1.063512  0.332883
  two  0.069877  0.246674 -0.011862  1.004812  1.327195
5 one -2.359419 -0.199543 -1.541996 -0.970736 -1.307030
6 two -0.919262 -1.549106  0.022185  0.758363 -0.660524

![len](https://gitee.com/zhao_long/image-store/raw/master/img/20201116210719.png)

### 根据索引层级分组

分层索引低的数据集有一个非常方便的结果，就是**能够在轴索引的某个层级上进行聚合**

In [63]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)

In [64]:
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.862580 -0.010032  0.050009  0.670216  0.852965
1     -0.955869 -0.023493 -2.304234 -0.652469 -1.218302
2     -1.332610  1.074623  0.723642  0.690002  1.001543
3     -0.503087 -0.622274 -0.921169 -0.726213  0.222896

In [66]:
hier_df.groupby( axis=1, level=1 ).sum()

tenor,1,3,5
0,1.532796,0.842933,0.050009
1,-1.608338,-1.241795,-2.304234
2,-0.642608,2.076166,0.723642
3,-1.229301,-0.399379,-0.921169


## 数据聚合

**聚合是指所有根据数组产生标量值的数据转换过程**

之前使用的`mean, sum, min,count`等

**可以使用自行制定的聚合**， **并调用已经在分组对象上定义好的方法**

要使用自己的聚合函数，需要将函数传递给`aggregate`或`agg`方法

In [67]:
df

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


In [68]:
grouped = df.groupby('key1')

In [69]:
grouped['data1'].quantile(0.9)

key1
a    1.668413
b   -0.523068
Name: data1, dtype: float64

In [72]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [73]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,2.170488,1.300498
b,0.036292,0.487276


![计算](https://gitee.com/zhao_long/image-store/raw/master/img/20201116213455.png)

In [74]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.746672  1.109736 -0.204708  0.137118  0.478943  1.222362   
b      2.0 -0.537585  0.025662 -0.555730 -0.546657 -0.537585 -0.528512   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.965781   3.0  0.910916  0.712217  0.092908  0.669671  1.246435   
b    -0.519439   2.0  0.525384  0.344556  0.281746  0.403565  0.525384   

                          
           75%       max  
key1                      
a     1.319920  1.393406  
b     0.647203  0.769023

### 逐列及多函数应用

上面的是**所有列进行聚合是使用aggregate 和 所需函数**

**根据各列同时使用多个函数进行聚合**

In [75]:
tips = pd.read_csv('pydata-book/examples/tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [76]:
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [77]:
grouped = tips.groupby(['day', 'smoker'])

In [79]:
grouped['tip_pct'].mean()

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [83]:
grouped['tip_pct'].agg(['mean', 'std', 'count', peak_to_peak ])

mean       std  count  peak_to_peak
day  smoker                                         
Fri  No      0.151650  0.028123      4      0.067349
     Yes     0.174783  0.051293     15      0.159925
Sat  No      0.158048  0.039767     45      0.235193
     Yes     0.147906  0.061375     42      0.290095
Sun  No      0.160113  0.042347     57      0.193226
     Yes     0.187250  0.154134     19      0.644685
Thur No      0.160298  0.038774     45      0.193350
     Yes     0.163863  0.039389     17      0.151240

In [84]:
grouped['tip_pct'].agg([('foo', 'mean'), ('bar','std'), ('bza','count'),('diff', peak_to_peak) ])

foo       bar  bza      diff
day  smoker                                   
Fri  No      0.151650  0.028123    4  0.067349
     Yes     0.174783  0.051293   15  0.159925
Sat  No      0.158048  0.039767   45  0.235193
     Yes     0.147906  0.061375   42  0.290095
Sun  No      0.160113  0.042347   57  0.193226
     Yes     0.187250  0.154134   19  0.644685
Thur No      0.160298  0.038774   45  0.193350
     Yes     0.163863  0.039389   17  0.151240

In [87]:
result = grouped[['tip_pct', 'total_bill']].agg(['count', 'mean', 'max'])
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [88]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [89]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped[['tip_pct', 'total_bill']].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [91]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})


tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [92]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],  'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### 返回不含行索引的聚合数据

`as_imndex=False`禁用分组键作为索引

In [94]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [96]:
tips.groupby(['day', 'smoker'], as_index=False).agg(['mean','count']).reset_index()

day smoker total_bill             tip            size         tip_pct  \
                     mean count      mean count      mean count      mean   
0   Fri     No  18.420000     4  2.812500     4  2.250000     4  0.151650   
1   Fri    Yes  16.813333    15  2.714000    15  2.066667    15  0.174783   
2   Sat     No  19.661778    45  3.102889    45  2.555556    45  0.158048   
3   Sat    Yes  21.276667    42  2.875476    42  2.476190    42  0.147906   
4   Sun     No  20.506667    57  3.167895    57  2.929825    57  0.160113   
5   Sun    Yes  24.120000    19  3.516842    19  2.578947    19  0.187250   
6  Thur     No  17.113111    45  2.673778    45  2.488889    45  0.160298   
7  Thur    Yes  19.190588    17  3.030000    17  2.352941    17  0.163863   

         
  count  
0     4  
1    15  
2    45  
3    42  
4    57  
5    19  
6    45  
7    17

## 应用：通用拆分-应用-联合

`GroupBy`最常见的目的是`apply`应用，**`apply`将对象分成很多块，然后在每一块上调用传递的函数，之后尝试将每一块连接到一起**

In [101]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [138]:
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [102]:
top(tips, 5)

,total_bill,tip,smoker,day,time,size,tip_pct
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [103]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [141]:
list(tips.groupby('smoker'))

[('No',
       total_bill   tip smoker   day    time  size   tip_pct
  0         16.99  1.01     No   Sun  Dinner     2  0.059447
  1         10.34  1.66     No   Sun  Dinner     3  0.160542
  2         21.01  3.50     No   Sun  Dinner     3  0.166587
  3         23.68  3.31     No   Sun  Dinner     2  0.139780
  4         24.59  3.61     No   Sun  Dinner     4  0.146808
  ..          ...   ...    ...   ...     ...   ...       ...
  235       10.07  1.25     No   Sat  Dinner     2  0.124131
  238       35.83  4.67     No   Sat  Dinner     3  0.130338
  239       29.03  5.92     No   Sat  Dinner     3  0.203927
  242       17.82  1.75     No   Sat  Dinner     2  0.098204
  243       18.78  3.00     No  Thur  Dinner     2  0.159744
  
  [151 rows x 7 columns]),
 ('Yes',
       total_bill   tip smoker  day    time  size   tip_pct
  56        38.01  3.00    Yes  Sat  Dinner     4  0.078927
  58        11.24  1.76    Yes  Sat  Dinner     2  0.156584
  60        20.29  3.21    Yes  Sat  Dinn

In [106]:
tips.groupby(['smoker', 'day']).apply(top, n=2,column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  91        22.49   3.50     No   Fri  Dinner     2  0.155625
            94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  59        48.27   6.73     No   Sat  Dinner     4  0.139424
            212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  112       38.07   4.00     No   Sun  Dinner     3  0.105070
            156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 85        34.83   5.17     No  Thur   Lunch     4  0.148435
            142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  90        28.97   3.00    Yes   Fri  Dinner     2  0.103555
            95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  102       44.30   2.50    Yes   Sat  Dinner     3  0.056433
            170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  184       40.55   3.00    Yes   Sun  Dinner     2  0.073983
            182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 83        32.68   5.00    Yes  Thur   Lunch     2  0.152999
            197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [152]:
tips.groupby(['smoker', 'day']).agg(top, n=2,column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  91        22.49   3.50     No   Fri  Dinner     2  0.155625
            94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  59        48.27   6.73     No   Sat  Dinner     4  0.139424
            212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  112       38.07   4.00     No   Sun  Dinner     3  0.105070
            156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 85        34.83   5.17     No  Thur   Lunch     4  0.148435
            142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  90        28.97   3.00    Yes   Fri  Dinner     2  0.103555
            95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  102       44.30   2.50    Yes   Sat  Dinner     3  0.056433
            170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  184       40.55   3.00    Yes   Sun  Dinner     2  0.073983
            182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 83        32.68   5.00    Yes  Thur   Lunch     2  0.152999
            197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [159]:
tips.groupby(['smoker', 'day']).count()

total_bill  tip  time  size  tip_pct
smoker day                                       
No     Fri            4    4     4     4        4
       Sat           45   45    45    45       45
       Sun           57   57    57    57       57
       Thur          45   45    45    45       45
Yes    Fri           15   15    15    15       15
       Sat           42   42    42    42       42
       Sun           19   19    19    19       19
       Thur          17   17    17    17       17

In [107]:
tips.groupby(['smoker', 'day']).apply(top, n=2,column='total_bill').index

MultiIndex([( 'No',  'Fri',  91),
            ( 'No',  'Fri',  94),
            ( 'No',  'Sat',  59),
            ( 'No',  'Sat', 212),
            ( 'No',  'Sun', 112),
            ( 'No',  'Sun', 156),
            ( 'No', 'Thur',  85),
            ( 'No', 'Thur', 142),
            ('Yes',  'Fri',  90),
            ('Yes',  'Fri',  95),
            ('Yes',  'Sat', 102),
            ('Yes',  'Sat', 170),
            ('Yes',  'Sun', 184),
            ('Yes',  'Sun', 182),
            ('Yes', 'Thur',  83),
            ('Yes', 'Thur', 197)],
           names=['smoker', 'day', None])

In [109]:
result = tips.groupby('smoker')['tip_pct'].describe()

In [110]:
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [111]:
result.unstack()

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [112]:
f = lambda x: x.describe()
grouped.apply(f)

total_bill       tip  size   tip_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

In [113]:
grouped.apply(top)

total_bill   tip    time  size   tip_pct
day  smoker                                              
Fri  No     99        12.46  1.50  Dinner     2  0.120385
            94        22.75  3.25  Dinner     2  0.142857
            91        22.49  3.50  Dinner     2  0.155625
            223       15.98  3.00   Lunch     3  0.187735
     Yes    226       10.09  2.00   Lunch     2  0.198216
...                     ...   ...     ...   ...       ...
Thur Yes    204       20.53  4.00   Lunch     4  0.194837
            205       16.47  3.23   Lunch     3  0.196114
            191       19.81  4.19   Lunch     2  0.211509
            200       18.71  4.00   Lunch     3  0.213789
            194       16.58  4.00   Lunch     2  0.241255

[39 rows x 5 columns]

In [114]:
grouped.agg(top)

total_bill   tip    time  size   tip_pct
day  smoker                                              
Fri  No     99        12.46  1.50  Dinner     2  0.120385
            94        22.75  3.25  Dinner     2  0.142857
            91        22.49  3.50  Dinner     2  0.155625
            223       15.98  3.00   Lunch     3  0.187735
     Yes    226       10.09  2.00   Lunch     2  0.198216
...                     ...   ...     ...   ...       ...
Thur Yes    204       20.53  4.00   Lunch     4  0.194837
            205       16.47  3.23   Lunch     3  0.196114
            191       19.81  4.19   Lunch     2  0.211509
            200       18.71  4.00   Lunch     3  0.213789
            194       16.58  4.00   Lunch     2  0.241255

[39 rows x 5 columns]

In [115]:
tips.groupby(['smoker', 'day']).agg(top, n=2,column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  91        22.49   3.50     No   Fri  Dinner     2  0.155625
            94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  59        48.27   6.73     No   Sat  Dinner     4  0.139424
            212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  112       38.07   4.00     No   Sun  Dinner     3  0.105070
            156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 85        34.83   5.17     No  Thur   Lunch     4  0.148435
            142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  90        28.97   3.00    Yes   Fri  Dinner     2  0.103555
            95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  102       44.30   2.50    Yes   Sat  Dinner     3  0.056433
            170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  184       40.55   3.00    Yes   Sun  Dinner     2  0.073983
            182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 83        32.68   5.00    Yes  Thur   Lunch     2  0.152999
            197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [118]:
grouped.apply?

In [144]:
grouped.agg?

### 压缩分组键

In [120]:
tips.groupby(['smoker', 'day'], group_keys=False).agg(top, n=2,column='total_bill')

,total_bill,tip,smoker,day,time,size,tip_pct
91,22.49,3.50,No,Fri,Dinner,2,0.155625
94,22.75,3.25,No,Fri,Dinner,2,0.142857
59,48.27,6.73,No,Sat,Dinner,4,0.139424
212,48.33,9.00,No,Sat,Dinner,4,0.186220
112,38.07,4.00,No,Sun,Dinner,3,0.105070
156,48.17,5.00,No,Sun,Dinner,6,0.103799
85,34.83,5.17,No,Thur,Lunch,4,0.148435
142,41.19,5.00,No,Thur,Lunch,5,0.121389
90,28.97,3.00,Yes,Fri,Dinner,2,0.103555
95,40.17,4.73,Yes,Fri,Dinner,4,0.117750


### 分位数与桶分析

`cut`或`qcut`与`groupby`一起是使用可以对**数据集更方便地分析或分桶分析**

In [121]:
frame = pd.DataFrame( {
    'data1': np.random.randn(1000),
    'data2': np.random.randn(1000),    
} )
frame

,data1,data2
0,0.051316,-0.732599
1,-1.157719,0.137889
2,0.816707,-0.719672
3,0.433610,1.927640
4,1.010737,-0.315813
...,...,...
995,-0.395813,-0.682273
996,-0.180737,-1.038534
997,1.337508,0.635111
998,-0.416584,-0.600026


In [124]:
quartiles = pd.cut(frame.data1, 4)
quartiles

0      (-1.23, 0.489]
1      (-1.23, 0.489]
2      (0.489, 2.208]
3      (-1.23, 0.489]
4      (0.489, 2.208]
            ...      
995    (-1.23, 0.489]
996    (-1.23, 0.489]
997    (0.489, 2.208]
998    (-1.23, 0.489]
999    (-1.23, 0.489]
Name: data1, Length: 1000, dtype: category
Categories (4, interval[float64]): [(-2.956, -1.23] < (-1.23, 0.489] < (0.489, 2.208] < (2.208, 3.928]]

In [125]:
def get_stats(group):
    return { 'min': group.min(), 'max': group.max(), 'count': group.count(), 'mean': group.mean()}

In [126]:
grouped = frame.data2.groupby(quartiles)

In [127]:
grouped.apply(get_stats)

data1                 
(-2.956, -1.23]  min       -3.399312
                 max        1.670835
                 count     94.000000
                 mean      -0.043940
(-1.23, 0.489]   min       -2.989741
                 max        3.260383
                 count    599.000000
                 mean      -0.011459
(0.489, 2.208]   min       -3.745356
                 max        2.954439
                 count    296.000000
                 mean       0.092068
(2.208, 3.928]   min       -1.929776
                 max        1.765640
                 count     11.000000
                 mean       0.055158
Name: data2, dtype: float64

In [128]:
grouped.agg(get_stats)

data1
(-2.956, -1.23]    {'min': -3.399311818813086, 'max': 1.670835397...
(-1.23, 0.489]     {'min': -2.9897409897464446, 'max': 3.26038307...
(0.489, 2.208]     {'min': -3.7453558773311113, 'max': 2.95443945...
(2.208, 3.928]     {'min': -1.9297764576212835, 'max': 1.76564043...
Name: data2, dtype: object

In [129]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-2.956, -1.23]",-3.399312,1.670835,94.0,-0.043940
"(-1.23, 0.489]",-2.989741,3.260383,599.0,-0.011459
"(0.489, 2.208]",-3.745356,2.954439,296.0,0.092068
"(2.208, 3.928]",-1.929776,1.765640,11.0,0.055158


In [132]:
grouped.agg(['min', 'max'])

,min,max
data1,,
"(-2.956, -1.23]",-3.399312,1.670835
"(-1.23, 0.489]",-2.989741,3.260383
"(0.489, 2.208]",-3.745356,2.954439
"(2.208, 3.928]",-1.929776,1.765640


In [135]:
grouping = pd.qcut( frame.data1, 10, labels=False )
grouping

0      5
1      1
2      8
3      6
4      8
      ..
995    3
996    4
997    9
998    3
999    6
Name: data1, Length: 1000, dtype: int64

In [137]:
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-3.399312,1.670835,100.0,-0.022372
1,-2.153545,2.628441,100.0,-0.045403
2,-2.925113,2.527939,100.0,-0.085406
3,-2.315555,3.260383,100.0,0.073851
4,-2.047939,2.074345,100.0,-0.120305
5,-2.989741,2.184810,100.0,0.036851
6,-2.084231,2.458842,100.0,0.001262
7,-3.056990,2.954439,100.0,-0.006403
8,-3.745356,2.735527,100.0,0.120793


###  使用指定分组值填充缺失值

在清除缺失值时，有时会使用`dropna`来**去除缺失值**，但是有时**可能想要使用修正值或来自于其他数据的值来输入到Null值**， `fillna`是一个正确工具

In [160]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan

In [161]:
s

0         NaN
1    1.141487
2         NaN
3    0.125036
4         NaN
5   -0.884727
dtype: float64

In [163]:
s.fillna(s.mean())

0    0.127265
1    1.141487
2    0.127265
3    0.125036
4    0.127265
5   -0.884727
dtype: float64

In [165]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
states

['Ohio',
 'New York',
 'Vermont',
 'Florida',
 'Oregon',
 'Nevada',
 'California',
 'Idaho']

In [166]:
group_key = ['East'] * 4 + ['West'] * 4
group_key

['East', 'East', 'East', 'East', 'West', 'West', 'West', 'West']

In [167]:
data = pd.Series(np.random.randn(8), index=states)
data

Ohio         -0.026877
New York      0.441817
Vermont       1.003998
Florida       0.682841
Oregon        0.326045
Nevada        0.425384
California   -3.428254
Idaho        -0.296336
dtype: float64

In [168]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data


Ohio         -0.026877
New York      0.441817
Vermont            NaN
Florida       0.682841
Oregon        0.326045
Nevada             NaN
California   -3.428254
Idaho              NaN
dtype: float64

In [169]:
data.groupby(group_key).mean()

East    0.365927
West   -1.551104
dtype: float64

In [170]:
fill_mean = lambda g: g.fillna(g.mean())

In [171]:
data.groupby(group_key).apply(fill_mean)

Ohio         -0.026877
New York      0.441817
Vermont       0.365927
Florida       0.682841
Oregon        0.326045
Nevada       -1.551104
California   -3.428254
Idaho        -1.551104
dtype: float64

In [172]:
fill_values = {'East': 0.5, 'West': -1}

In [173]:
fill_func = lambda g:g.fillna(fill_values[g.name] )

In [174]:
data.groupby(group_key).apply(fill_func)

Ohio         -0.026877
New York      0.441817
Vermont       0.500000
Florida       0.682841
Oregon        0.326045
Nevada       -1.000000
California   -3.428254
Idaho        -1.000000
dtype: float64

### 随机采样和排列

In [183]:
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

In [184]:
cards

['AH',
 '2H',
 '3H',
 '4H',
 '5H',
 '6H',
 '7H',
 '8H',
 '9H',
 '10H',
 'JH',
 'KH',
 'QH',
 'AS',
 '2S',
 '3S',
 '4S',
 '5S',
 '6S',
 '7S',
 '8S',
 '9S',
 '10S',
 'JS',
 'KS',
 'QS',
 'AC',
 '2C',
 '3C',
 '4C',
 '5C',
 '6C',
 '7C',
 '8C',
 '9C',
 '10C',
 'JC',
 'KC',
 'QC',
 'AD',
 '2D',
 '3D',
 '4D',
 '5D',
 '6D',
 '7D',
 '8D',
 '9D',
 '10D',
 'JD',
 'KD',
 'QD']

In [185]:
deck = pd.Series(card_val, index=cards)
deck.head()

AH    1
2H    2
3H    3
4H    4
5H    5
dtype: int64

In [186]:
def draw(deck, n=5):
    return deck.sample(n)
draw(deck)

AD      1
10C    10
4D      4
QD     10
3C      3
dtype: int64

In [188]:
get_suit = lambda card: card[-1]
deck.groupby(get_suit).apply(draw, n=2)

C  2C     2
   4C     4
D  KD    10
   JD    10
H  AH     1
   5H     5
S  5S     5
   6S     6
dtype: int64

In [189]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

7C     7
KC    10
5D     5
2D     2
3H     3
7H     7
9S     9
2S     2
dtype: int64

### 分组加权平均和相关性

在`groupby`的拆分-应用-联合的范式下，**DataFrame的列间操作或两个Series之间的操作，例如 加权平均是可以做到的**

In [190]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
df

,category,data,weights
0,a,-2.206538,0.671150
1,a,-0.040652,0.938971
2,a,-0.125891,0.626735
3,a,-0.213543,0.598730
4,b,1.280423,0.271266
5,b,0.342845,0.750880
6,b,2.068144,0.993182
7,b,0.821758,0.953179


In [191]:
grouped = df.groupby('category')

In [194]:
get_wavg = lambda g: np.average( g['data'] , weights=g['weights'])

In [195]:
grouped.apply(get_wavg)

category
a   -0.608638
b    1.159538
dtype: float64

In [196]:
close_px = pd.read_csv('pydata-book/examples/stock_px_2.csv', parse_dates=True, index_col=0)
close_px

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01
2003-01-07,7.43,21.93,28.95,922.93
2003-01-08,7.28,21.31,28.83,909.93
...,...,...,...,...
2011-10-10,388.81,26.94,76.28,1194.89
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66


In [198]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [199]:
spx_corr = lambda x: x.corrwith(x['SPX'])

In [200]:
rets = close_px.pct_change().dropna()

In [201]:
get_year = lambda x: x.year

In [202]:
by_year = rets.groupby(get_year)

In [203]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [205]:
by_year.apply(lambda x: x['AAPL'].corr(x['SPX']))

2003    0.541124
2004    0.374283
2005    0.467540
2006    0.428267
2007    0.508118
2008    0.681434
2009    0.707103
2010    0.710105
2011    0.691931
dtype: float64

### 逐组线性回归

In [207]:
pip install statsmodels

Looking in indexes: https://pypi.doubanio.com/simple
     |████████████████████████████████| 9.5 MB 352 kB/s eta 0:00:01
     |████████████████████████████████| 231 kB 475 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [208]:
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [209]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## 数据透视表与交叉表

数据透视表时电子表格程序和其他数据分析软件中**常见的数据汇总工具**

**根据一个或多个键聚合一张表的数据**， **将数据在矩形格式中排列，其中一些分组键是沿着行的，另一些分组键是沿着列的**

`pandas`中的透视表是**由groupby和分级索引的重塑操作实现的**

`pd.pivot_table`函数或者`DataFrame.pivot_table`方法，除了**为`groupby`提供一个方便的接口，还可以添加部分总计，也称作“边距”**

In [210]:
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [228]:
tips['size'].groupby(tips['smoker']).mean()

smoker
No     2.668874
Yes    2.408602
Name: size, dtype: float64

In [229]:
tips['size'].groupby(tips['day']).mean()

day
Fri     2.105263
Sat     2.517241
Sun     2.842105
Thur    2.451613
Name: size, dtype: float64

In [226]:
tips.mean()

total_bill    19.785943
tip            2.998279
size           2.569672
tip_pct        0.160803
dtype: float64

In [211]:
tips.groupby(['day', 'smoker']).mean()

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

In [212]:
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [213]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'])

size   tip_pct
time   day                     
Dinner Fri   2.166667  0.158916
       Sat   2.517241  0.153152
       Sun   2.842105  0.166897
       Thur  2.000000  0.159744
Lunch  Fri   2.000000  0.188765
       Thur  2.459016  0.161301

In [214]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [223]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker').mean()


         smoker
size     No        2.497563
         Yes       2.292727
tip_pct  No        0.160929
         Yes       0.170661
dtype: float64

In [225]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [216]:
tips.groupby( ['time', 'day', 'smoker'])[['size', 'tip_pct']].mean().unstack()

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [231]:
tips.groupby(['time', 'day'])[['size', 'tip_pct']].mean()

size   tip_pct
time   day                     
Dinner Fri   2.166667  0.158916
       Sat   2.517241  0.153152
       Sun   2.842105  0.166897
       Thur  2.000000  0.159744
Lunch  Fri   2.000000  0.188765
       Thur  2.459016  0.161301

In [232]:
tips.groupby(['smoker'])[['size', 'tip_pct']].mean()

,size,tip_pct
smoker,,
No,2.668874,0.159328
Yes,2.408602,0.163196


In [233]:
tips.pivot_table('tip_pct', index=['time', 'day'], columns='smoker', aggfunc=len, margins=True)

smoker          No   Yes    All
time   day                     
Dinner Fri     3.0   9.0   12.0
       Sat    45.0  42.0   87.0
       Sun    57.0  19.0   76.0
       Thur    1.0   NaN    1.0
Lunch  Fri     1.0   6.0    7.0
       Thur   44.0  17.0   61.0
All          151.0  93.0  244.0

In [234]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
...                      ...       ...       ...       ...
Lunch  3    Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

[21 rows x 4 columns]

### 交叉表

交叉表`crosstab`是数据透视表的一种特殊情况，计**算的是分组中的频率**

In [236]:
from io import StringIO
data = """\
Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep='\s+')

In [237]:
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [238]:
pd.crosstab( data.Nationality, data.Handedness, margins=True )

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [241]:
data.groupby(['Nationality','Handedness']).count()

Sample
Nationality Handedness          
Japan       Left-handed        2
            Right-handed       3
USA         Left-handed        1
            Right-handed       4

In [243]:
data.pivot_table(index='Nationality', columns='Handedness', aggfunc=len)

Sample             
Handedness  Left-handed Right-handed
Nationality                         
Japan                 2            3
USA                   1            4

In [245]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244